In [36]:
# =========================
# Установка библиотек
# =========================
!pip install catboost -q

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool
from google.colab import files

# =========================
# 1. Загрузка данных
# =========================

train_path = 'rent_train_preprocessed.xlsx'
test_path  = 'rent_test_preprocessed.xlsx'

train = pd.read_excel(train_path)
test  = pd.read_excel(test_path)

print('Train shape:', train.shape)
print('Test shape :', test.shape)
print('Train columns:\n', train.columns.tolist())

# =========================
# 2. Целевая и ID
# =========================

if 'Price' not in train.columns:
    raise ValueError("Столбец 'Price' не найден в train!")

target_col = 'Price'

if 'ID  объявления' in train.columns:
    id_col_source = 'ID  объявления'
elif 'ID' in train.columns:
    id_col_source = 'ID'
else:
    id_col_source = train.columns[0]

print('Target column      :', target_col)
print('ID column (source) :', id_col_source)

# =========================
# 3. Отбор признаков
# =========================
# Убираем тяжёлые текстовые поля, которые только шумят
drop_cols = [
    target_col,
    id_col_source,
    'Адрес',
    'Описание',
    'Дополнительно'
]
drop_cols = [c for c in drop_cols if c in train.columns]

X = train.drop(columns=drop_cols).copy()
y = train[target_col].copy()

# Для теста берём те же признаки
X_test = test[X.columns].copy()
test_ids = test[id_col_source].copy()

print('Используемых признаков:', len(X.columns))

# =========================
# 4. Обработка типов и пропусков
# =========================

# Категориальные признаки: object
cat_features = [col for col in X.columns if X[col].dtype == 'object']
# Числовые признаки
num_features = [col for col in X.columns if col not in cat_features]

print('Категориальных признаков:', len(cat_features))
print('Числовых признаков      :', len(num_features))

# ЧИСЛОВЫЕ: заполняем медианой
for col in num_features:
    if X[col].isna().any():
        med = X[col].median()
        X[col] = X[col].fillna(med)
        X_test[col] = X_test[col].fillna(med)

# КАТЕГОРИАЛЬНЫЕ: переводим в строки + 'missing'
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

    X[col] = X[col].fillna('missing')
    X_test[col] = X_test[col].fillna('missing')

# Индексы категориальных признаков для CatBoost
cat_features_idx = [X.columns.get_loc(col) for col in cat_features]

# =========================
# 5. Лог-преобразование таргета
# =========================

y_log = np.log1p(y)   # log(1 + Price)

# =========================
# 6. Train/valid split
# =========================

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y_log,
    test_size=0.2,
    random_state=42
)

train_pool = Pool(X_train, y_train, cat_features=cat_features_idx)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features_idx)

# =========================
# 7. Обучение модели
# =========================

model = CatBoostRegressor(
    loss_function='RMSE',     # работаем в лог-пространстве
    eval_metric='RMSE',
    depth=8,
    learning_rate=0.03,
    n_estimators=3000,
    random_seed=42,
    verbose=200,
    task_type='CPU'
)

model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

# =========================
# 8. Оценка MAE (в рублях)
# =========================

valid_pred_log = model.predict(valid_pool)
valid_pred = np.expm1(valid_pred_log)   # обратно из логов
y_valid_original = np.expm1(y_valid)

mae = mean_absolute_error(y_valid_original, valid_pred)
print(f'\nValidation MAE: {mae:,.0f} руб.')

# =========================
# 9. Обучение на всём train и предсказание на test
# =========================

full_train_pool = Pool(X, y_log, cat_features=cat_features_idx)
model_full = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    depth=8,
    learning_rate=0.03,
    n_estimators=3000,
    random_seed=42,
    verbose=200,
    task_type='CPU'
)

model_full.fit(full_train_pool)

test_pool = Pool(X_test, cat_features=cat_features_idx)
test_pred_log = model_full.predict(test_pool)
test_pred = np.expm1(test_pred_log)

# =========================
# 10. Формирование submission
# =========================

submission = pd.DataFrame({
    'ID': test_ids.values,   # Kaggle ждёт колонку 'ID'
    'Price': test_pred
})

submission_file = 'submission.csv'
submission.to_csv(submission_file, index=False)
print(f'\nФайл {submission_file} сохранён')
print(submission.head())

# =========================
# 11. Скачивание submission
# =========================

files.download(submission_file)


Train shape: (1017, 31)
Test shape : (255, 30)
Train columns:
 ['ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Price', 'Комнаты', 'Площадь_общая', 'Этаж', 'Этажность', 'Метро_станция', 'Метро_минуты', 'Метро_тип', 'lat', 'lon']
Target column      : Price
ID column (source) : ID  объявления
Используемых признаков: 26
Категориальных признаков: 18
Числовых признаков      : 8
0:	learn: 0.9225541	test: 0.9662188	best: 0.9662188 (0)	total: 78.7ms	remaining: 3m 56s
200:	learn: 0.2022725	test: 0.2995845	best: 0.2995845 (200)	total: 5.33s	remaining: 1m 14s
400:	learn: 0.1515359	test: 0.2830224	best: 0.2830224 (400)	total: 10.2s	remaining: 1m 6s
600:	learn: 0.1222720	test: 0.2785043	best: 0.2784553 (595)	total: 14.8s	remaining: 59.2s
800:	learn: 0.0995220

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>